<a href="https://colab.research.google.com/github/rastringer/promptcraft/blob/main/chaining_prompts.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chaining prompts

Chaining inputs and outputs.

In [ ]:
# Install the packages
! pip install --upgrade google-cloud-aiplatform
! pip install shapely<2.0.0

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

If you're on Colab, run the following cell to authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai
from vertexai.preview.language_models import ChatModel, InputOutputTextPair

# Replace the project and location placeholder values below
vertexai.init(project="<your-project-id>", location="<location>")
chat_model = ChatModel.from_pretrained("chat-bison@001")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}

We will switch to a json file soon. For now, here's our `products` text again.

In [ ]:
products = """
name: Caffeino Classic
category: Espresso Machines
brand: EliteBrew
model_number: EB-1001
warranty: 2 years
rating: 4.6/5 stars
features:
  15-bar pump for authentic espresso extraction.
  Milk frother for creating creamy cappuccinos and lattes.
  Removable water reservoir for easy refilling.
description: The Caffeino Classic by EliteBrew is a powerful espresso machine that delivers rich and flavorful shots of espresso with the convenience of a built-in milk frother, perfect for indulging in your favorite cafe-style beverages at home.
price: £179.99

name: BeanPresso
category: Single Serve Coffee Makers
brand: FreshBrew
model_number: FB-500
warranty: 1 year
rating: 4.3/5 stars
features:
  Compact design ideal for small spaces or travel.
  Compatible with various coffee pods for quick and easy brewing.
  Auto-off feature for energy efficiency and safety.
description: The BeanPresso by FreshBrew is a compact single-serve coffee maker that allows you to enjoy a fresh cup of coffee effortlessly using your favorite coffee pods, making it the perfect companion for those with limited space or always on the go.
price: £49.99

name: BrewBlend Pro
category: Drip Coffee Makers
brand: MasterRoast
model_number: MR-800
warranty: 3 years
rating: 4.7/5 stars
features:
  Adjustable brew strength for customized coffee flavor.
  Large LCD display with programmable timer for convenient brewing.
  Anti-drip system to prevent messes on the warming plate.
description: The BrewBlend Pro by MasterRoast offers a superior brewing experience with adjustable brew strength, programmable timer, and anti-drip system, ensuring a perfectly brewed cup of coffee every time, making mornings more enjoyable.
price: £89.99

name: SteamGenie
category: Stovetop Coffee Makers
brand: KitchenWiz
model_number: KW-200
warranty: 2 years
rating: 4.4/5 stars
features:
  Classic Italian stovetop design for rich and aromatic coffee.
  Durable stainless steel construction for long-lasting performance.
  Available in multiple sizes to suit different brewing needs.
description: The SteamGenie by KitchenWiz is a traditional stovetop coffee maker that harnesses the essence of Italian coffee culture, crafted with durable stainless steel and delivering a rich, authentic coffee experience with every brew.
price: £39.99

name: AeroBlend Max
category: Coffee and Espresso Combo Machines
brand: AeroGen
model_number: AG-1200
warranty: 2 years
rating: 4.9/5 stars
features:
  Dual-functionality for brewing coffee and espresso.
  Built-in burr grinder for fresh coffee grounds.
  Adjustable temperature and brew strength settings for personalized beverages.
description: The AeroBlend Max by AeroGen is a versatile coffee and espresso combo machine that combines the convenience of brewing both coffee and espresso with a built-in grinder,
allowing you to enjoy the perfect cup of your preferred caffeinated delight with ease.
price: £299.99
"""

As in earlier notebooks, delimiters help us isolate the inputs and responses.

Here, we give the model specific to output recommendations as a python dictionary, which will help with post-processing tasks (eg adding to a shopping cart).

We also give clear guidelines about the products and categories the model can return. This helps minimize the risk of the model hallucinating coffee machines not part of our catalogue.

In [ ]:
delimiter = "####"
context = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a python dictionary of objects, where each object has \
the following format:
    'category': <one of Espresso Machines, \
    Single Serve Coffee Makers, \
    Drip Coffee Makers, \
    Stovetop Coffee Makers,
    Coffee and Espresso Combo Machines>,
AND
    'products': <a list of products that must \
    be found in the allowed products below>

For example,
  'category': 'Coffee and Espresso Combo Machines', 'products': ['AeroBlend Max'],

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products:

Espresso Machines category:
Caffeino Classic

Single Serve Coffee Makers:
BeanPresso

Drip Coffee Makers:
BrewBlend Pro

Stovetop Coffee Makers:
SteamGenie

Coffee and Espresso Combo Machines:
AeroBlend Max

Only output the list of objects, with nothing else.
"""


In [ ]:
user_message_1 = f"""
I'd like info about the SteamGenie and the BrewBlend Pro. \
"""

chat = chat_model.start_chat(
    context=context,
    examples=[]
)

response = chat.send_message(user_message_1, **parameters)
print(response.text)

[{'category': 'Stovetop Coffee Makers', 'products': ['SteamGenie']}, {'category': 'Drip Coffee Makers', 'products': ['BrewBlend Pro']}]


Though it looks like a Python dictionary, our response is a TextGenerationResponse object, so we have a few more steps to convert it into a dict we can use.

In [ ]:
type(response)

vertexai.language_models._language_models.TextGenerationResponse

In [ ]:
temp_str = str(response)

In [ ]:
temp_str

"[{'category': 'Stovetop Coffee Makers', 'products': ['SteamGenie']}, {'category': 'Drip Coffee Makers', 'products': ['BrewBlend Pro']}]"

### Products json

Switching from our products string to json will allow us to do more with results

In [ ]:
products = {
    "Caffeino Classic": {
      "name": "Caffeino Classic",
      "category": "Espresso Machines",
      "brand": "EliteBrew",
      "model_number": "EB-1001",
      "warranty": "2 years",
      "rating": "4.6/5 stars",
      "features": [
        "15-bar pump for authentic espresso extraction.",
        "Milk frother for creating creamy cappuccinos and lattes.",
        "Removable water reservoir for easy refilling."
      ],
      "description": "The Caffeino Classic by EliteBrew is a powerful espresso machine that delivers rich and flavorful shots of espresso with the convenience of a built-in milk frother, perfect for indulging in your favorite cafe-style beverages at home.",
      "price": "£179.99"
    },
    "BeanPresso": {
      "name": "BeanPresso",
      "category": "Single Serve Coffee Makers",
      "brand": "FreshBrew",
      "model_number": "FB-500",
      "warranty": "1 year",
      "rating": "4.3/5 stars",
      "features": [
        "Compact design ideal for small spaces or travel.",
        "Compatible with various coffee pods for quick and easy brewing.",
        "Auto-off feature for energy efficiency and safety."
      ],
      "description": "The BeanPresso by FreshBrew is a compact single-serve coffee maker that allows you to enjoy a fresh cup of coffee effortlessly using your favorite coffee pods, making it the perfect companion for those with limited space or always on the go.",
      "price": "£49.99"
    },
    "BrewBlend Pro": {
      "name": "BrewBlend Pro",
      "category": "Drip Coffee Makers",
      "brand": "MasterRoast",
      "model_number": "MR-800",
      "warranty": "3 years",
      "rating": "4.7/5 stars",
      "features": [
        "Adjustable brew strength for customized coffee flavor.",
        "Large LCD display with programmable timer for convenient brewing.",
        "Anti-drip system to prevent messes on the warming plate."
      ],
      "description": "The BrewBlend Pro by MasterRoast offers a superior brewing experience with adjustable brew strength, programmable timer, and anti-drip system, ensuring a perfectly brewed cup of coffee every time, making mornings more enjoyable.",
      "price": "£89.99"
    },
    "SteamGenie": {
      "name": "SteamGenie",
      "category": "Stovetop Coffee Makers",
      "brand": "KitchenWiz",
      "model_number": "KW-200",
      "warranty": "2 years",
      "rating": "4.4/5 stars",
      "features": [
        "Classic Italian stovetop design for rich and aromatic coffee.",
        "Durable stainless steel construction for long-lasting performance.",
        "Available in multiple sizes to suit different brewing needs."
      ],
      "description": "The SteamGenie by KitchenWiz is a traditional stovetop coffee maker that harnesses the essence of Italian coffee culture, crafted with durable stainless steel and delivering a rich, authentic coffee experience with every brew.",
      "price": "£39.99"
    },
    "AeroBlend Max": {
      "name": "AeroBlend Max",
      "category": "Coffee and Espresso Combo Machines",
      "brand": "AeroGen",
      "model_number": "AG-1200",
      "warranty": "2 years",
      "rating": "4.9/5 stars",
      "features": [
        "Dual-functionality for brewing coffee and espresso.",
        "Built-in burr grinder for fresh coffee grounds.",
        "Adjustable temperature and brew strength settings for personalized beverages."
      ],
      "description": "The AeroBlend Max by AeroGen is a versatile coffee and espresso combo machine that combines the convenience of brewing both coffee and espresso with a built-in grinder, allowing you to enjoy the perfect cup of your preferred caffeinated delight with ease.",
      "price": "£299.99"
    }
}

In [ ]:
def get_products():
    return products

### Read Python string into Python list of dictionaries

In [ ]:
import json

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None

In [ ]:
category_and_product_list = read_string_to_list(temp_str)
print(category_and_product_list)

[{'category': 'Stovetop Coffee Makers', 'products': ['SteamGenie']}, {'category': 'Drip Coffee Makers', 'products': ['BrewBlend Pro']}]


### Helper functions

Now that our products are in json, we can use various helper functions to render responses into a format more useful than text. For example, we can check the model's outputs are relevant, or pass the items and their details on to a shopping cart.

#### Note:

These helper functions are from DeepLearning AI's *Building Systems with the ChatGPT API* course.

In [ ]:
def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

In [ ]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string

In [ ]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "name": "SteamGenie",
    "category": "Stovetop Coffee Makers",
    "brand": "KitchenWiz",
    "model_number": "KW-200",
    "warranty": "2 years",
    "rating": "4.4/5 stars",
    "features": [
        "Classic Italian stovetop design for rich and aromatic coffee.",
        "Durable stainless steel construction for long-lasting performance.",
        "Available in multiple sizes to suit different brewing needs."
    ],
    "description": "The SteamGenie by KitchenWiz is a traditional stovetop coffee maker that harnesses the essence of Italian coffee culture, crafted with durable stainless steel and delivering a rich, authentic coffee experience with every brew.",
    "price": "\u00a339.99"
}
{
    "name": "BrewBlend Pro",
    "category": "Drip Coffee Makers",
    "brand": "MasterRoast",
    "model_number": "MR-800",
    "warranty": "3 years",
    "rating": "4.7/5 stars",
    "features": [
        "Adjustable brew strength for customized coffee flavor.",
        "Large LCD displa

In [ ]:
context = f"""
You're a customer service assistant for a coffee shop's \
e-commerce site. Our product list can be found in {products}. Respond in a friendly and professional \
tone with concise answers. \
Please ask the user relevant follow-up questions.
"""

user_message_1 = f"""
Tell me about the Brew Blend pro and \
the stovetop coffee maker. \
Also do you have an espresso machine?"""

chat = chat_model.start_chat(
    context=context,
    examples=[]
)

assistant_response = chat.send_message(f"""{user_message_1}{product_information_for_user_message_1}""", **parameters)
print(assistant_response)

The BrewBlend Pro is a drip coffee maker that offers a superior brewing experience with adjustable brew strength, programmable timer, and anti-drip system. The SteamGenie is a stovetop coffee maker that brews coffee by passing hot water over ground coffee beans. We also have an espresso machine, the AeroBlend Max, which is a versatile coffee and espresso combo machine that combines the convenience of brewing both coffee and espresso with a built-in grinder.


### Check output

Now that we have our outputs as handly lists and strings, we can add them as inputs for the model to check. This step will become less necessary as models become more sophisticated, and is only recommended for extremely highly sensitive applications since adds cost and latency and may be unnecessary

In [ ]:
context = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""
customer_message = f"""
Tell me all about the Brew Blend pro and \
the stovetop coffee maker - features and pricing. \
Also do you have an espresso machine?"""

q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information_for_user_message_1}```
Agent response: ```{assistant_response}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""

chat = chat_model.start_chat(
    context=context,
    examples=[]
)

response = chat.send_message(f"""{q_a_pair}""")
print(response)

Y
